# Energy Cost Optimization Tool

This tool leverages data from the U.S. Energy Information Administration (EIA) API along with insights generated by a Large Language Model (LLM) API. The goal is to analyze and optimize energy costs using real-time data and advanced language processing for actionable insights.

In [ ]:
from interface import EnergyCostOptimizationInterface
import sys

# Check if running in Google Colab and clone the repository
if 'google.colab' in sys.modules:
    # Clone the repository and navigate to the folder
    !git clone https://github.com/your-username/Energy-Cost-Optimization-Tool.git
    %cd Energy-Cost-Optimization-Tool

# Set up the environment and import the necessary modules
try:
    from interface import EnergyCostOptimizationInterface
except ModuleNotFoundError:
    # If imports fail (because it's the first run in Colab), restart the kernel to recognize new modules
    print("Restarting kernel to load cloned files...")
    import os
    os._exit(0)

# Initialize and display the interface
EnergyCostOptimizationInterface()